In [ ]:
"""
WHISPER FINE-TUNING NOTEBOOK
Input: train.jsonl + eval.jsonl from data processing pipeline
Output: Fine-tuned Whisper model + WER comparison (baseline vs fine-tuned)

Requirements:
pip install transformers datasets evaluate jiwer librosa soundfile torch accelerate
"""

import json
import torch
import librosa
import numpy as np
from pathlib import Path
from typing import Dict, List
from dataclasses import dataclass
from datasets import Dataset, DatasetDict, Audio
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from jiwer import wer as compute_wer
import warnings
warnings.filterwarnings("ignore")


# ============================================================================
# CONFIG
# ============================================================================

CONFIG = {
    "dataset_dir": Path("output/whisper_children_dataset/training_dataset"),
    "model_name": "openai/whisper-base",
    "output_dir": Path("output/whisper_finetuned"),
    "num_train_epochs": 3,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "learning_rate": 1e-5,
    "warmup_steps": 500,
    "weight_decay": 0.01,
    "logging_steps": 100,
    "save_steps": 500,
    "eval_steps": 500,
    "language": "French",
    "task": "transcribe"
}


# ============================================================================
# ZONE 1: LOAD DATASETS FROM JSONL
# ============================================================================

def load_datasets_from_jsonl(dataset_dir: Path) -> DatasetDict:
    """Load train and eval datasets from JSONL files"""
    
    print("="*70)
    print("📂 STEP 1: LOAD DATASETS FROM JSONL")
    print("="*70)
    
    train_file = dataset_dir / "train.jsonl"
    eval_file = dataset_dir / "eval.jsonl"
    
    if not train_file.exists() or not eval_file.exists():
        raise FileNotFoundError(f"Missing JSONL files in {dataset_dir}")
    
    # Load JSONL files
    def load_jsonl(path):
        data = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                data.append(json.loads(line))
        return data
    
    train_data = load_jsonl(train_file)
    eval_data = load_jsonl(eval_file)
    
    print(f"\n✓ Loaded {len(train_data)} training samples")
    print(f"✓ Loaded {len(eval_data)} evaluation samples")
    
    # Create Hugging Face Datasets
    train_dataset = Dataset.from_dict({
        "audio": [d["audio"] for d in train_data],
        "text": [d["text"] for d in train_data],
    })
    
    eval_dataset = Dataset.from_dict({
        "audio": [d["audio"] for d in eval_data],
        "text": [d["text"] for d in eval_data],
    })
    
    # Cast to Audio type (Hugging Face will load audio files)
    train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
    eval_dataset = eval_dataset.cast_column("audio", Audio(sampling_rate=16000))
    
    datasets = DatasetDict({
        "train": train_dataset,
        "eval": eval_dataset
    })
    
    print(f"\n✅ Datasets created with audio loading\n")
    
    return datasets


# ============================================================================
# ZONE 2: PREPARE DATA FOR TRAINING
# ============================================================================

def prepare_dataset(batch, processor, language="French", task="transcribe"):
    """Prepare audio and text for training"""
    
    # Load and process audio
    audio = batch["audio"]
    
    # Whisper expects audio to be resampled to 16kHz (already done by Audio())
    # Extract mel spectrogram
    inputs = processor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        language=language
    )
    
    # Tokenize text
    with processor.as_target_processor():
        labels = processor(batch["text"], language=language)
    
    batch["input_features"] = inputs.input_features[0]
    batch["labels"] = labels.input_ids
    
    return batch


def prepare_datasets(datasets: DatasetDict, processor, config: Dict) -> DatasetDict:
    """Prepare all datasets for training"""
    
    print("="*70)
    print("🔧 STEP 2: PREPARE DATASETS")
    print("="*70)
    
    print("\n  Preparing training dataset...", end=" ")
    train_dataset = datasets["train"].map(
        lambda batch: prepare_dataset(batch, processor, config["language"], config["task"]),
        remove_columns=["audio", "text"],
        num_proc=4
    )
    print("✓")
    
    print("  Preparing evaluation dataset...", end=" ")
    eval_dataset = datasets["eval"].map(
        lambda batch: prepare_dataset(batch, processor, config["language"], config["task"]),
        remove_columns=["audio", "text"],
        num_proc=4
    )
    print("✓")
    
    prepared = DatasetDict({
        "train": train_dataset,
        "eval": eval_dataset
    })
    
    print(f"\n✅ Datasets prepared\n")
    
    return prepared


# ============================================================================
# ZONE 3: FINE-TUNING
# ============================================================================

def finetune_whisper(prepared_datasets: DatasetDict, processor, config: Dict) -> WhisperForConditionalGeneration:
    """Fine-tune Whisper model"""
    
    print("="*70)
    print("🎓 STEP 3: FINE-TUNE WHISPER")
    print("="*70)
    
    # Load model
    print(f"\n  Loading model: {config['model_name']}...", end=" ")
    model = WhisperForConditionalGeneration.from_pretrained(config["model_name"])
    print("✓")
    
    # Freeze encoder (optional, speeds up training)
    model.encoder.requires_grad_(False)
    print("  Encoder frozen (optional)")
    
    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=str(config["output_dir"]),
        per_device_train_batch_size=config["per_device_train_batch_size"],
        per_device_eval_batch_size=config["per_device_eval_batch_size"],
        gradient_accumulation_steps=1,
        learning_rate=config["learning_rate"],
        warmup_steps=config["warmup_steps"],
        num_train_epochs=config["num_train_epochs"],
        weight_decay=config["weight_decay"],
        logging_steps=config["logging_steps"],
        save_steps=config["save_steps"],
        eval_steps=config["eval_steps"],
        save_total_limit=2,
        evaluation_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        predict_with_generate=True,
        generation_max_length=225,
        fp16=torch.cuda.is_available(),
        gradient_checkpointing=True,
        dataloader_pin_memory=False,
        remove_unused_columns=False,
        label_names=["labels"],
    )
    
    # Trainer
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=prepared_datasets["train"],
        eval_dataset=prepared_datasets["eval"],
        data_collator=DataCollatorSpeechSeq2SeqWithPadding(processor),
        compute_metrics=lambda x: compute_metrics(x, processor),
        tokenizer=processor.tokenizer,
    )
    
    print("\n🚀 Starting training...\n")
    
    # Fine-tune
    trainer.train()
    
    print("\n✅ Fine-tuning completed\n")
    
    return model


class DataCollatorSpeechSeq2SeqWithPadding:
    """Data collator for Whisper"""
    
    def __init__(self, processor):
        self.processor = processor
    
    def __call__(self, features):
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )
        
        batch["labels"] = labels
        
        return batch


def compute_metrics(pred, processor):
    """Compute WER metric during training"""
    
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # Replace -100 with processor's pad token id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    
    # Decode predictions and references
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)
    
    # Compute WER
    wer = compute_wer(label_str, pred_str)
    
    return {"wer": wer}


# ============================================================================
# ZONE 4: EVALUATE - BASELINE VS FINE-TUNED
# ============================================================================

@dataclass
class EvaluationResult:
    segment_id: str
    ground_truth: str
    baseline_prediction: str
    finetuned_prediction: str
    baseline_wer: float
    finetuned_wer: float
    improvement: float


class WhisperEvaluator:
    """Evaluate baseline and fine-tuned models"""
    
    def __init__(self, processor, baseline_model, finetuned_model):
        self.processor = processor
        self.baseline_model = baseline_model
        self.finetuned_model = finetuned_model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.baseline_model.to(self.device)
        self.finetuned_model.to(self.device)
        self.baseline_model.eval()
        self.finetuned_model.eval()
    
    def transcribe(self, audio_path: Path, model) -> str:
        """Transcribe with a model"""
        
        try:
            # Load audio
            audio, sr = librosa.load(str(audio_path), sr=16000)
            
            # Prepare input
            inputs = self.processor(
                audio,
                sampling_rate=16000,
                language="French",
                return_tensors="pt"
            )
            input_features = inputs.input_features.to(self.device)
            
            # Generate
            with torch.no_grad():
                predicted_ids = model.generate(input_features, language="<|fr|>", task="transcribe")
            
            # Decode
            transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
            
            return transcription
        except Exception as e:
            print(f"  ⚠️  Error: {audio_path.name} - {e}")
            return ""
    
    def evaluate_dataset(self, dataset: Dataset, batch_size: int = 50) -> List[EvaluationResult]:
        """Evaluate baseline vs fine-tuned on dataset"""
        
        results = []
        
        print(f"\n🔄 Evaluating {len(dataset)} samples\n")
        
        for i, sample in enumerate(dataset):
            audio_path = Path(sample["audio"])
            ground_truth = sample["text"]
            
            # Get predictions from both models
            baseline_pred = self.transcribe(audio_path, self.baseline_model)
            finetuned_pred = self.transcribe(audio_path, self.finetuned_model)
            
            # Calculate WER
            baseline_wer = compute_wer(ground_truth, baseline_pred)
            finetuned_wer = compute_wer(ground_truth, finetuned_pred)
            improvement = baseline_wer - finetuned_wer
            
            results.append(EvaluationResult(
                segment_id=f"sample_{i:05d}",
                ground_truth=ground_truth,
                baseline_prediction=baseline_pred,
                finetuned_prediction=finetuned_pred,
                baseline_wer=baseline_wer,
                finetuned_wer=finetuned_wer,
                improvement=improvement
            ))
            
            if (i + 1) % batch_size == 0:
                avg_baseline = sum(r.baseline_wer for r in results) / len(results)
                avg_finetuned = sum(r.finetuned_wer for r in results) / len(results)
                print(f"  ✓ {i + 1}/{len(dataset)} | Baseline WER: {avg_baseline:.3f} | Fine-tuned WER: {avg_finetuned:.3f}")
        
        return results


def print_evaluation_report(results: List[EvaluationResult]):
    """Print detailed evaluation report"""
    
    baseline_wers = [r.baseline_wer for r in results]
    finetuned_wers = [r.finetuned_wer for r in results]
    improvements = [r.improvement for r in results]
    
    print("\n" + "="*70)
    print("📊 STEP 4: EVALUATION REPORT - BASELINE vs FINE-TUNED")
    print("="*70)
    
    print(f"\n📈 Statistics:")
    print(f"   Total segments: {len(results)}")
    
    print(f"\n   BASELINE Model:")
    print(f"      Avg WER:     {sum(baseline_wers) / len(baseline_wers):.3f}")
    print(f"      Min WER:     {min(baseline_wers):.3f}")
    print(f"      Max WER:     {max(baseline_wers):.3f}")
    print(f"      Median WER:  {sorted(baseline_wers)[len(baseline_wers)//2]:.3f}")
    
    print(f"\n   FINE-TUNED Model:")
    print(f"      Avg WER:     {sum(finetuned_wers) / len(finetuned_wers):.3f}")
    print(f"      Min WER:     {min(finetuned_wers):.3f}")
    print(f"      Max WER:     {max(finetuned_wers):.3f}")
    print(f"      Median WER:  {sorted(finetuned_wers)[len(finetuned_wers)//2]:.3f}")
    
    print(f"\n   IMPROVEMENT:")
    avg_improvement = sum(improvements) / len(improvements)
    print(f"      Avg improvement: {avg_improvement:.3f} WER points")
    print(f"      Relative improvement: {(avg_improvement / (sum(baseline_wers) / len(baseline_wers))) * 100:.1f}%")
    
    # Distribution
    print(f"\n   Better performance:")
    better = sum(1 for imp in improvements if imp > 0)
    worse = sum(1 for imp in improvements if imp < 0)
    same = len(improvements) - better - worse
    print(f"      Improved:  {better} ({better/len(improvements)*100:.1f}%)")
    print(f"      Worse:     {worse} ({worse/len(improvements)*100:.1f}%)")
    print(f"      Same:      {same} ({same/len(improvements)*100:.1f}%)")
    
    # Examples
    print(f"\n📝 Best improvements:")
    best = sorted(results, key=lambda x: x.improvement, reverse=True)[:3]
    for i, r in enumerate(best, 1):
        print(f"      {i}. {r.ground_truth}")
        print(f"         Baseline: {r.baseline_prediction} (WER: {r.baseline_wer:.3f})")
        print(f"         Fine-tuned: {r.finetuned_prediction} (WER: {r.finetuned_wer:.3f})")
        print(f"         Improvement: {r.improvement:.3f}")
    
    print("\n" + "="*70 + "\n")


def save_evaluation_results(results: List[EvaluationResult], output_dir: Path):
    """Save evaluation results to JSON"""
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    data = [
        {
            "segment_id": r.segment_id,
            "ground_truth": r.ground_truth,
            "baseline_prediction": r.baseline_prediction,
            "finetuned_prediction": r.finetuned_prediction,
            "baseline_wer": r.baseline_wer,
            "finetuned_wer": r.finetuned_wer,
            "improvement": r.improvement
        }
        for r in results
    ]
    
    with open(output_dir / "evaluation_results.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Evaluation results saved: {output_dir / 'evaluation_results.json'}\n")


# ============================================================================
# MAIN: RUN COMPLETE FINE-TUNING PIPELINE
# ============================================================================

def run_finetuning_pipeline():
    """Run complete fine-tuning pipeline"""
    
    print("\n" + "="*70)
    print("🚀 WHISPER FINE-TUNING PIPELINE")
    print("="*70 + "\n")
    
    # ZONE 1: Load datasets
    datasets = load_datasets_from_jsonl(CONFIG["dataset_dir"])
    
    # Load processor
    print("="*70)
    print("📦 LOADING PROCESSOR")
    print("="*70)
    print(f"\n  Loading processor from {CONFIG['model_name']}...", end=" ")
    processor = WhisperProcessor.from_pretrained(CONFIG["model_name"])
    print("✓\n")
    
    # ZONE 2: Prepare datasets
    prepared_datasets = prepare_datasets(datasets, processor, CONFIG)
    
    # ZONE 3: Fine-tune
    finetuned_model = finetune_whisper(prepared_datasets, processor, CONFIG)
    
    # Save fine-tuned model
    print("="*70)
    print("💾 SAVE FINE-TUNED MODEL")
    print("="*70)
    print(f"\n  Saving model...", end=" ")
    finetuned_model.save_pretrained(CONFIG["output_dir"] / "model")
    processor.save_pretrained(CONFIG["output_dir"] / "processor")
    print("✓\n")
    
    # ZONE 4: Evaluate
    print("="*70)
    print("🔍 COMPARING BASELINE vs FINE-TUNED")
    print("="*70)
    
    # Load baseline
    print("\n  Loading baseline model...", end=" ")
    baseline_model = WhisperForConditionalGeneration.from_pretrained(CONFIG["model_name"])
    print("✓")
    
    evaluator = WhisperEvaluator(processor, baseline_model, finetuned_model)
    
    # Evaluate on eval dataset
    results = evaluator.evaluate_dataset(datasets["eval"], batch_size=50)
    
    # Print report
    print_evaluation_report(results)
    
    # Save results
    save_evaluation_results(results, CONFIG["output_dir"])
    
    print("="*70)
    print("✅ FINE-TUNING PIPELINE COMPLETE")
    print("="*70)
    print(f"\n📂 Output directory: {CONFIG['output_dir']}")
    print("\n📋 Generated files:")
    print(f"   model/ ........................ Fine-tuned model")
    print(f"   processor/ ................... Processor config")
    print(f"   evaluation_results.json ...... Detailed results")
    print("\n🎓 Fine-tuned model ready for deployment!\n")


if __name__ == "__main__":
    run_finetuning_pipeline()